In [4]:
from utils import extract_image_info, split_channels, segment_nuclei, extract_intensities, classify_cells, convert_labels_to_edges
from cellpose import models
import napari
from pathlib import Path
import pandas as pd

# -------- DEFINE YOUR ANALYSIS PARAMETERS BELOW --------- #

# This is the default Cellpose model to segment nuclei, it leverages a GPU if its available
model = models.Cellpose(gpu=True, model_type="nuclei")

# Diameter in pixels of the nuclei present in your image, helps Cellpose to adjust nuclei mask predictions
cellpose_nuclei_diameter = 40

# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 5

# Any nuclei containing a higher than threshold value for signal intensity will be considered as a + cell for said marker
h2a_threshold = 40 # Mean intensity within the nucleus mask (h2a signal)
cfos_threshold = 20 # Mean intensity within the nucleus mask (cfos signal)

# Define the image path of the image you want to explore
image = Path("data\IHC_1\AD1867\CA1\Image4.lsm")

#TODO: Display the stack in Napari

# ------- ANALYSIS PIPELINE --------- #

# Extract filename, region, mouse and IHC round
filename, region, mouse_id, ihc_round = extract_image_info(image)

# Split channels
nuclei_img, h2a_img, cfos_img = split_channels(image)

# Segment nuclei
nuclei_masks = segment_nuclei(nuclei_img, gaussian_sigma, model, cellpose_nuclei_diameter)

# Morphological and intensity measurements
merged_df = extract_intensities(nuclei_masks, h2a_img, cfos_img, filename, region, mouse_id, ihc_round)

# Select H2A and CFOS positive cells based on mean_intensity thresholds, return a mask of + cells
h2a_nuclei_labels, cfos_nuclei_labels, double_pos_nuclei_labels, h2a_pos_labels, cfos_pos_labels, double_pos_labels = classify_cells(merged_df, nuclei_masks, h2a_threshold, cfos_threshold)

# Visualize nuclei labels as just the perimeter for ease of interpretation    
nuclei_edges = convert_labels_to_edges(nuclei_masks, radius=1)
h2a_nuclei_edges = convert_labels_to_edges(h2a_nuclei_labels, radius=1)
cfos_nuclei_edges = convert_labels_to_edges(cfos_nuclei_labels, radius=1)
double_pos_nuclei_edges = convert_labels_to_edges(double_pos_nuclei_labels, radius=1)

# Initialize napari.Viewer and display input stacks and label processing steps
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(nuclei_img)
viewer.add_image(h2a_img)
viewer.add_image(cfos_img)
viewer.add_labels(nuclei_edges)
viewer.add_labels(h2a_nuclei_edges)
viewer.add_labels(cfos_nuclei_edges)
viewer.add_labels(double_pos_nuclei_edges)

# Show the Dataframe containing the per label information
# Set display options
pd.set_option('display.max_rows', None)  # Show all rows
merged_df

,filename,region,mouse_id,ihc_round,label,cfos_intensity_mean,cfos_intensity_max,h2a_intensity_mean,h2a_intensity_max,area_filled,perimeter,equivalent_diameter
0,Image4,CA1,AD1867,IHC_1,1,4.572680,17.0,3.283713,22.0,571.0,108.183766,26.963304
1,Image4,CA1,AD1867,IHC_1,2,8.817259,16.0,3.137056,15.0,197.0,62.384776,15.837556
2,Image4,CA1,AD1867,IHC_1,3,18.592821,43.0,85.877734,255.0,1783.0,173.195959,47.646470
3,Image4,CA1,AD1867,IHC_1,4,43.948468,73.0,114.359331,216.0,718.0,103.639610,30.235509
4,Image4,CA1,AD1867,IHC_1,5,77.656460,127.0,103.987304,227.0,1339.0,150.468037,41.290044
5,Image4,CA1,AD1867,IHC_1,6,30.801498,56.0,16.647940,69.0,534.0,85.254834,26.075082
6,Image4,CA1,AD1867,IHC_1,7,17.796425,47.0,14.800397,63.0,1007.0,119.639610,35.807153
7,Image4,CA1,AD1867,IHC_1,8,5.046832,19.0,2.856061,38.0,1452.0,143.154329,42.997021
8,Image4,CA1,AD1867,IHC_1,9,12.597554,29.0,59.854333,251.0,1881.0,194.208153,48.938365
9,Image4,CA1,AD1867,IHC_1,10,25.863095,43.0,37.422619,99.0,168.0,46.284271,14.625466
